What-If Tool on COMPAS
Copyright 2019 Google LLC. SPDX-License-Identifier: Apache-2.0

This notebook shows use of the [What-If Tool](https://pair-code.github.io/what-if-tool) on the COMPAS dataset.

For ML fairness background on COMPAS see:
- https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing
- https://www.propublica.org/article/how-we-analyzed-the-compas-recidivism-algorithm
- http://www.crj.org/assets/2017/07/9_Machine_bias_rejoinder.pdf

The dataset is from the [COMPAS kaggle page](https://www.kaggle.com/danofer/compass).

This notebook trains a linear classifier on the on the COMPAS dataset to mimic the behavior of the the COMPAS recidivism classifier. We can then analyze our COMPAS proxy model for fairness using the What-If Tool.

The specific binary classification task for this model is to determine if a person belongs in the "Low" risk class according to COMPAS (negative class), or the "Medium" or "High" risk class (positive class).

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import functools
import matplotlib.pyplot as plt

## Define helper functions

In [2]:
# Creates a tf feature spec from the dataframe and columns specified.
def create_feature_spec(df, columns=None):
    feature_spec = {}
    if columns == None:
        columns = df.columns.values.tolist()
    for f in columns:
        if df[f].dtype is np.dtype(np.int64):
            feature_spec[f] = tf.io.FixedLenFeature(shape=(), dtype=tf.int64)
        elif df[f].dtype is np.dtype(np.float64):
            feature_spec[f] = tf.io.FixedLenFeature(shape=(), dtype=tf.float32)
        else:
            feature_spec[f] = tf.io.FixedLenFeature(shape=(), dtype=tf.string)
    return feature_spec

In [3]:
# Creates simple numeric and categorical feature columns from a feature spec and a
# list of columns from that spec to use.
#
# NOTE: Models might perform better with some feature engineering such as bucketed
# numeric columns and hash-bucket/embedding columns for categorical features.
def create_feature_columns(columns, feature_spec):
    ret = []
    for col in columns:
        if feature_spec[col].dtype is tf.int64 or feature_spec[col].dtype is tf.float32:
            ret.append(tf.feature_column.numeric_column(col))
        else:
            ret.append(tf.feature_column.indicator_column(
                tf.feature_column.categorical_column_with_vocabulary_list(col, list(df[col].unique()))))
    return ret

In [4]:
# An input function for providing input to a model from tf.Examples
def tfexamples_input_fn(examples, feature_spec, label, mode=tf.estimator.ModeKeys.EVAL,
                       num_epochs=None, 
                       batch_size=64):
    def ex_generator():
        for i in range(len(examples)):
            yield examples[i].SerializeToString()
    dataset = tf.data.Dataset.from_generator(
      ex_generator, tf.dtypes.string, tf.TensorShape([]))
    if mode == tf.estimator.ModeKeys.TRAIN:
        dataset = dataset.shuffle(buffer_size=2 * batch_size + 1)
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(lambda tf_example: parse_tf_example(tf_example, label, feature_spec))
    dataset = dataset.repeat(num_epochs)
    return dataset

In [5]:
# Parses Tf.Example protos into features for the input function.
def parse_tf_example(example_proto, label, feature_spec):
    parsed_features = tf.io.parse_example(serialized=example_proto, features=feature_spec)
    target = parsed_features.pop(label)
    return parsed_features, target

In [6]:
# Converts a dataframe into a list of tf.Example protos.
def df_to_examples(df, columns=None):
    examples = []
    if columns == None:
        columns = df.columns.values.tolist()
    for index, row in df.iterrows():
        example = tf.train.Example()
        for col in columns:
            if df[col].dtype is np.dtype(np.int64):
                example.features.feature[col].int64_list.value.append(int(row[col]))
            elif df[col].dtype is np.dtype(np.float64):
                example.features.feature[col].float_list.value.append(row[col])
            elif row[col] == row[col]:
                example.features.feature[col].bytes_list.value.append(row[col].encode('utf-8'))
        examples.append(example)
    return examples

In [7]:
# Converts a dataframe column into a column of 0's and 1's based on the provided test.
# Used to force label columns to be numeric for binary classification using a TF estimator.
def make_label_column_numeric(df, label_column, test):
    df[label_column] = np.where(test(df[label_column]), 1, 0)

## Read training dataset from CSV

In [8]:
df = pd.read_csv('https://storage.googleapis.com/what-if-tool-resources/computefest2019/cox-violent-parsed_filt.csv')

In [9]:
df.shape

(18316, 40)

In [10]:
df.head(2)

,id,name,first,last,sex,dob,age,age_cat,race,juv_fel_count,...,vr_charge_desc,type_of_assessment,decile_score.1,score_text,screening_date,v_type_of_assessment,v_decile_score,v_score_text,priors_count.1,event
0,1.0,miguel hernandez,miguel,hernandez,Male,18/04/1947,69,Greater than 45,Other,0,...,NaN,Risk of Recidivism,1,Low,14/08/2013,Risk of Violence,1,Low,0,0
1,2.0,miguel hernandez,miguel,hernandez,Male,18/04/1947,69,Greater than 45,Other,0,...,NaN,Risk of Recidivism,1,Low,14/08/2013,Risk of Violence,1,Low,0,0


## Specify input columns and columns to predict

In [11]:
# Filter out entries with no indication of recidivism or no compass score
df = df[df['is_recid'] != -1]
df = df[df['decile_score'] != -1]

# Rename recidivism column
df['recidivism_within_2_years'] = df['is_recid']

# Make the COMPASS label column numeric (0 and 1), for use in our model
df['COMPASS_determination'] = np.where(df['score_text'] == 'Low', 0, 1)

# Set column to predict
label_column = 'COMPASS_determination'

# Get list of all columns from the dataset we will use for model input or output.
input_features = ['sex', 'age', 'race', 'priors_count', 'juv_fel_count', 'juv_misd_count', 'juv_other_count']
features_and_labels = input_features + [label_column]

features_for_file = input_features + ['recidivism_within_2_years', 'COMPASS_determination']

## Convert dataset to tf.Example protos

In [12]:
examples = df_to_examples(df, features_for_file)

## Create and train the classifier

In [13]:
num_steps = 2000 #@param {type: "number"}

In [14]:
# Create a feature spec for the classifier
feature_spec = create_feature_spec(df, features_and_labels)

In [15]:
# Define and train the classifier
train_inpf = functools.partial(tfexamples_input_fn, examples, feature_spec, label_column)
classifier = tf.estimator.LinearClassifier(
    feature_columns=create_feature_columns(input_features, feature_spec))
classifier.train(train_inpf, steps=num_steps)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/k7/pvgyxzd171bdd36dggbbxmrr0000gn/T/tmpe3jbl6dw', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variab

/Users/cinny/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/k7/pvgyxzd171bdd36dggbbxmrr0000gn/T/tmpe3jbl6dw/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 0.6931472, step = 0
INFO:tensorflow:global_step/sec: 132.1
INFO:tensorflow:loss = 0.55241585, step = 100 (0.758 sec)
INFO:tensorflow:global_step/sec: 132.308
INFO:tensorflow:loss = 0.4251055, step = 200 (0.756 sec)
INFO:tensorflow:global_step/sec: 120.548
INFO:tensorflow:loss = 0.5206141, step = 300 (0.830 sec)
INFO:tensorflow:global_step/sec: 121.895
INFO:tensorflow:loss = 0.5033277, 

## Model Evaluation
-- Cinny's prelim eval of the baseline model

In [17]:
def get_data(df, truth_label='recidivism_within_2_years', pred_label='COMPASS_determination'):
    tp = df[(df[truth_label]==1) & (df[pred_label]==1)]
    tn = df[(df[truth_label]==0) & (df[pred_label]==0)]
    fp = df[(df[truth_label]==0) & (df[pred_label]==1)]
    fn = df[(df[truth_label]==1) & (df[pred_label]==0)]
    return tp, tn, fp, fn

In [18]:
def get_length(tp, tn, fp, fn):
    return len(tp), len(tn), len(fp), len(fn)

In [19]:
def get_accuracy(df, truth_label='recidivism_within_2_years', pred_label='COMPASS_determination'):
    tp, tn, fp, fn = get_data(df)
    TP, TN, FP, FN = get_length(tp, tn, fp, fn)
    return (TP+TN)/(TP+FP+FN+TN)

In [20]:
def get_precision(df, truth_label='recidivism_within_2_years', pred_label='COMPASS_determination'):
    tp, tn, fp, fn = get_data(df)
    TP, TN, FP, FN = get_length(tp, tn, fp, fn)
    return (TP)/(TP+FP)

In [21]:
def get_recall(df, truth_label='recidivism_within_2_years', pred_label='COMPASS_determination'):
    tp, tn, fp, fn = get_data(df)
    TP, TN, FP, FN = get_length(tp, tn, fp, fn)
    return (TP)/(TP+FN)

In [22]:
def get_f1(df, truth_label='recidivism_within_2_years', pred_label='COMPASS_determination'):
    P = get_precision(df)
    R = get_recall(df)
    return 2*(P*R)/(P+R)

### Accuracy
`Accuracy = (TP+TN)/(TP+FP+FN+TN)`

Accuracy is a valid choice of evaluation for classification problems which are well balanced and not skewed or No class imbalance.

In [23]:
get_accuracy(df)

0.6261799874134676

### Precision
`Precision = (TP)/(TP+FP)`

Precision is a valid choice of evaluation metric when we want to be very sure of our prediction.

In [24]:
get_precision(df)

0.6002136752136752

### Recall

`Recall = (TP)/(TP+FN)`

Recall is a valid choice of evaluation metric when we want to capture as many positives as possible.

In [25]:
get_recall(df)

0.6680142687277051

### F1 Score

`F1 = 2 * (precision * recall) / (precision + recall)`

The F1 score is a number between 0 (worst) and 1 (best). It is used when you want your model to have both good precision and recall.

### F_beta

`F_beta = (1 + beta^2) * (precision * recall) / ( (beta^2 * precision) + recall )`

The F1 score gives equal weight to precision and recall. `beta` means we give `beta` times more importance to recall as precision.

In [26]:
get_f1(df)

0.6323016319639843

## Bias Assessment

In [47]:
df.groupby(['race', 'recidivism_within_2_years', 'COMPASS_determination']).count()['id']

race              recidivism_within_2_years  COMPASS_determination
African-American  0                          0                        1212
                                             1                        1421
                  1                          0                         746
                                             1                        2264
Asian             0                          0                          23
                                             1                           6
                  1                          0                           2
                                             1                           8
Caucasian         0                          0                        1410
                                             1                         629
                  1                          0                         653
                                             1                         823
Hispanic          0              

In [49]:
tp, tn, fp, fn = get_data(df)
TP, TN, FP, FN = get_length(tp, tn, fp, fn)

In [50]:
#(TP+TN)/(TP+FP+FN+TN)
get_accuracy(df)

0.6261799874134676

In [51]:
(2264+1212)/(2264+1421+746+1212) #African American

0.6159844054580896

In [52]:
(823+1410)/(823+629+653+1410) #Caucasian

0.6352773826458037